---
title: '전공별 직업 분포 확인'
author: "Jinii"
format: 
  dashboard:
    nav-buttons:
      - icon: github
        href: https://github.com/Jinii1/lsbigdata-project1
---


## {.sidebar}


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from shiny import ui, reactive, render

# 데이터 불러오기
df = pd.read_csv('../..//data/Common_Jobs_by_Major.csv')

# 필요한 열만 선택
df = df[['Detailed Occupation', 'Total Population', 'ID CIP2']]

# CIP2 코드 숫자를 한글 전공명으로 변경
df['ID CIP2'] = df['ID CIP2'].replace({
    14: '공학',
    19: '인문과학',
    27: '수학_통계',
    31: '공원_레크리에이션_레저',
    26: '생물학'
})

# Detailed Occupation 변수를 한글 직업군으로 변경
df['Detailed Occupation'] = df['Detailed Occupation'].replace({
    'Other managers': '기타경영자',
    'Software developers': '소프트웨어개발자',
    'Civil engineers': '토목기사',
    'Miscellaneous engineers, including nuclear engineers': '기타기술자',
    'Physicians': '의료인'
})

# 사이드바에 추가할 직업군 리스트
occupations = df['Detailed Occupation'].unique()

# 사이드바 입력 정의
ui.input_checkbox_group(
    "occupations", "Select Occupations:", 
    choices=occupations, selected=occupations
)

In [ ]:
@reactive.Calc
def filtered_data():
    selected_occupations = input.occupations()
    if not selected_occupations:
        return pd.DataFrame()
    return df[df['Detailed Occupation'].isin(selected_occupations)]

@render.plot
def occupation_pie_chart():
    df_filtered = filtered_data()
    if df_filtered.empty:
        return go.Figure()  # 빈 데이터프레임일 경우 빈 그림 반환
    
    df_pie = df_filtered.groupby('Detailed Occupation').sum('Total Population').reset_index()
    
    fig = go.Figure()

    fig.add_trace(go.Pie(
        values=df_pie['Total Population'], 
        labels=df_pie['Detailed Occupation'],
        direction='clockwise', 
        sort=True
    ))

    fig.update_layout(title=dict(text="직업군별 인구 분포", x=0.5))
    return fig